# Symmetry reduction

**Adapted from**: [SymbolicWedderburn example](https://github.com/kalmarek/SymbolicWedderburn.jl/blob/tw/ex_sos/examples/ex_C4.jl)

In [1]:
import MutableArithmetics as MA
using MultivariatePolynomials
using MultivariateBases

using DynamicPolynomials
@polyvar x[1:4]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂, x₃, x₄],)

We would like to find the minimum value of the polynomial

In [2]:
poly = sum(x) + sum(x.^2)

x₄ + x₃ + x₂ + x₁ + x₄² + x₃² + x₂² + x₁²

As we can decouple the problem for each `x[i]` for which `x[i] + x[i]^2` has
minimum value 0.25, we would expect to get `-1` as answer.
Can this decoupling be exploited by SumOfSquares as well ?
For this, we need to use a certificate that can exploit the permutation symmetry of the polynomial.

In [3]:
using SumOfSquares

We define the symmetry group as a permutation group in the variables.
In order to do that, we define the action of a permutation on a monomial
as the monomial obtained after permuting the variables.

In [4]:
using PermutationGroups
G = PermGroup([perm"(1,2,3,4)"])

Permutation group on 1 generator generated by
 (1,2,3,4)

We can use this certificate as follows:

In [5]:
import Clarabel
solver = Clarabel.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
pattern = Symmetry.Pattern(G, Symmetry.VariablePermutation())
con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
optimize!(model)
value(t)

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 6
  constraints   = 20
  nnz(P)        = 0
  nnz(A)        = 48
  cones (total) = 3
    : Zero        = 1,  numel = 15
    : Nonnegative = 1,  numel = 2
    : SecondOrder = 1,  numel = 3

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter

-0.9999999598042424

We indeed find `-1`, let's verify that symmetry was exploited:

In [6]:
gram_matrix(con_ref)

BlockDiagonalGramMatrix with 3 blocks:
[1] Block with row/column basis:
     Simple basis:
  FixedBasis([1.0·1 + 0.0·x₄ + 0.0·x₃ + 0.0·x₂ + 0.0·x₁, 0.0·1 - 0.5·x₄ - 0.5·x₃ - 0.5·x₂ - 0.5·x₁])
    And entries in a 2×2 SymMatrix{Float64}:
      0.9999999598042425  -0.9999999999999999
     -0.9999999999999999   1.0000000000000002
[2] Block with row/column basis:
     Semisimple basis with 2 simple sub-bases:
  FixedBasis([0.0·1 - 0.7071067811865472·x₄ + 0.0·x₃ + 0.7071067811865475·x₂ + 0.0·x₁])
  FixedBasis([0.0·1 + 0.0·x₄ - 0.7071067811865472·x₃ + 0.0·x₂ + 0.7071067811865475·x₁])
    And entries in a 1×1 SymMatrix{Float64}:
     1.0000000000000004
[3] Block with row/column basis:
     Simple basis:
  FixedBasis([0.0·1 - 0.5·x₄ + 0.5·x₃ - 0.5·x₂ + 0.5·x₁])
    And entries in a 1×1 SymMatrix{Float64}:
     1.0000000000000002

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*